<a href="https://colab.research.google.com/github/sosucat/sbl-optimizer-colab/blob/main/sbl_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💡 Make a Copy to Edit

This notebook is **view-only**.  
To interact with it (e.g., upload your image, run cells, or save your own results), you’ll need to **make your own copy**:

- In the top-left corner:  **`File → Save a copy in Drive`**

- Your copy will be saved to your Google Drive.

- You’ll need to be signed into a Google account.


# 🐐 sbl-optimizer: Print Pattern Optimizer for [Swell by Light](https://sites.gatech.edu/futurefeelings/2025/03/07/swell-by-light-tei-25/)
[![PyPI version](https://badge.fury.io/py/sbl-optimizer.svg)](https://badge.fury.io/py/sbl-optimizer)
[![GitHub](https://img.shields.io/badge/GitHub_repo-black?logo=github)](https://github.com/sosucat/sbl-optimizer)

[![Homepage](https://img.shields.io/badge/🔗_Homepage-black)](https://sites.gatech.edu/futurefeelings/2025/03/07/swell-by-light-tei-25/)
[![Author](https://img.shields.io/badge/Author-black?logo=googlescholar&logoColor=white)](https://sosuke-ichihashi.com/)
[![Research paper](https://img.shields.io/badge/Research_Paper-black?logo=acm)](https://doi.org/10.1145/3689050.3704420)
[![Fabrication](https://img.shields.io/badge/🔗_Fabrication-black)](https://sites.gatech.edu/futurefeelings/2025/07/23/make-puffy-patterns-with-light/)
[![Watch fabrication demo on YouTube](https://img.shields.io/badge/Fabrication-750014?logo=youtube)](https://youtu.be/LomVS_jHxl0?feature=shared)

Welcome to **sbl-optimizer**, a tool optimizing your pattern images for 2.5D texture fabrication!
**This notebook is an advanced version** walking through all the codes.
If you just want to use sbl-optimizer without minding the coding, check out the simplified Colab notebook:

[![Simpler Colab](https://img.shields.io/badge/Simpler_Colab_Notebook-black?logo=googlecolab)](https://colab.research.google.com/drive/1Kpvq15wZrzsnQI28_JfkDSqCwT1ouyxj?usp=sharing)

---


![A printed pattern's shades change as the optimization progresses, and the resulting temperature distribution gets closer to the intended pattern.](https://sites.gatech.edu/futurefeelings/files/2025/03/opt_step.gif)

Optimization of the printed pattern results in a uniform temperature distribution (right) closely matching the original pattern (left). In this example, the leaves are over-heated while the stems are under-heated before the optimization. As the iteration number goes up, the leaves' temperature goes down while the stems' goes up, resulting in a more uniform temperature distribution.

---

## 📑 Table of Contents
1. [Setup & Installation](#scrollTo=vkd63QF86-g3)  
2. [Library & Function Imports](#scrollTo=mHjEnEtZ7azV)  
3. [Quick Start: Optimize Sample Pattern](#scrollTo=jEwPKC3COIbX)  
4. [Optimize Your Pattern](#scrollTo=ay2GXkzFPJuT)  
5. [Change Settings](#scrollTo=uHP6gd2SYqIg)  
6. [Troubleshooting & FAQ](#scrollTo=-pFXIm8gfsJX)
7. [Credits & License](#scrollTo=q2DhdMj88Kec)
8. [Next Step](#scrollTo=af-mVoGqfRIf)

---

## 1. Setup & Installation
> We install the package.

In [ ]:
# Install the latest version from PyPI
!pip install sbl-optimizer

## 2. Library & Function Imports

> We install necessary libraries and functions.

In [ ]:
import json
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import importlib.resources as pkg_resources
from google.colab import files
from IPython.display import IFrame, display, Javascript
import base64

from sbl_optimizer.config import Config
from sbl_optimizer.heat_solver import image_to_heat_pattern, optimize
from sbl_optimizer.utils import compute_dims
from sbl_optimizer.io import save_pattern, save_plots

##3. Quick Start: Optimize Sample Pattern
**Overview:** We walk through how to optimize patterns and download the results.
1. ⚙️ Load Default Settings
2. 🖼️ Load Sample Image
3. 🔧 Prepare for Optimization
4. 👀 Preview Prepared Image
5. 🔁 Run Optimization
6. 👀 Save & Preview Optimized Pattern
7. 👀 Show Swell Pattern & Temperature Distribution
8. 💾 Download Optimized Pattern
9. 📉 (Optional) Show Optimization Performance

### 3-1. ⚙️ Load Default Settings


> First, decide on the parameters to use for the optimization—such as the temperature at which the paste expands or the resolution of the simulation. For now, we'll stick with the default settings.



In [ ]:
# Locate the bundled config.json inside the installed package
def get_default_config_path():
    return pkg_resources.as_file(
        pkg_resources.files("sbl_optimizer.assets") / "config.json"
    )

with get_default_config_path() as config_path:
    with open(config_path) as f:
        cfg_data = json.load(f)

print("Default configuration:")
for k, v in cfg_data.items():
    print(f"- {k}: {v}")

# Create Config object
cfg = Config.from_file(config_path)

### 3-2. 🖼️ Load Sample Image
> This time, we'll focus on walking through the full process, so we'll use a sample image provided with the library.

In [ ]:
with pkg_resources.as_file(
    pkg_resources.files("sbl_optimizer.assets") / "sample.jpg"
) as sample_img_path:
    img = Image.open(sample_img_path)
    plt.figure(figsize=(4,4))
    plt.imshow(img)
    plt.axis('off')
    plt.title("Sample Input Image")
    plt.show()

### 3-3. 🔧 Prepare for Optimization


> We'll first make the patterned areas of the image a uniform gray. Then, we'll adjust the image resolution to match that of the simulation.


In [ ]:
mask, H_init, max_heat = image_to_heat_pattern(sample_img_path, cfg)

### 3-4. 👀 Preview Prepared Image
> Here is the preprocessed sample image. This represents the pre-optimization state.



In [ ]:
# Visualize the initialized print pattern using matplotlib
def visualize_init_pattern(H_init, max_heat):
  plt.figure(figsize=(4,4))
  plt.imshow(H_init/max_heat, cmap='gray_r', vmin=0.0, vmax=1.0)
  plt.axis('off')
  plt.title("Initial print pattern")
  plt.show()

visualize_init_pattern(H_init, max_heat)

### 3-5. 🔁 Run Optimization

> The goal of this optimization is to create a swell pattern that closely resembles the original design, even after heat conduction occurs within the paper. If we simply make the entire pattern uniformly gray or black, the wider lines tend to overheat while the thinner ones don’t heat up enough due to the heat conduction. As a result, the temperature distribution won't match the original image, and the heat-based swells won’t either. To address this, we perform “optimization” to adjust the shading in different areas so that the resulting temperature distribution, after heat conduction, matches the original image as closely as possible.

In [ ]:
%%time
# Set the maximum height of the output
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'))

# Compute physical dimensions (for logging)
img_w, img_h, phys_w, phys_h = compute_dims(Image.open(sample_img_path))

# Optimize
T_best, H_best, errors = optimize(mask, H_init, max_heat, img_w, img_h, phys_w, phys_h, cfg)
print(f"Optimization completed in {len(errors)} iterations.")

###3-6. 👀 Save and Preview Optimized Pattern

> We'll now save and display the optimized pattern image. If you're satisfied with the result, you can download the image later.



In [ ]:
# Save pattern PDF
pattern_pdf = save_pattern(sample_img_path, H_best, max_heat)
print(f"Saved optimized pattern to: {pattern_pdf}")

# Display the generated PDF inline in Colab
with open(pattern_pdf, "rb") as pdf_file:
    encoded_pdf = base64.b64encode(pdf_file.read()).decode('utf-8')
IFrame(f"data:application/pdf;base64,{encoded_pdf}", width=400, height=400)

### 3-7. 👀 Show Swell Pattern and Temperature Distribution

> See side-by-side visualizations of the simulated raised texture (swell pattern) and temperature across the sheet (temperature distribution).



In [ ]:
def display_plots(plots):
  fig, (ax_swell, ax_temperature) = plt.subplots(1, 2, figsize=(4 * 2, 4))

  img_plot = Image.open(plots[0])
  ax_swell.imshow(img_plot)
  ax_swell.axis('off')
  ax_swell.set_title('Swell Pattern')

  img_plot = Image.open(plots[1])
  ax_temperature.imshow(img_plot)
  ax_temperature.axis('off')
  ax_temperature.set_title('Temperature Distribution [C°]')

  plt.tight_layout()
  plt.show()

# Save and display plots with matplotlib
plots = save_plots(T_best, phys_w, phys_h, sample_img_path, dpi=72)
display_plots(plots)

### 3-8. 💾 Download Optimized Pattern

> Finally, grab the PDF file you just created. You’re ready to print and experiment!

In [ ]:
# Ask user if they want to download the PDF locally
def download_pdf(pdf_file):
  download_choice = input('Do you want to download the optimized PDF? (y/n): ').strip().lower()
  if download_choice == 'y':
      files.download(str(pdf_file))

download_pdf(pattern_pdf)

###3-9. 📉 (Optional) Show Optimization Performance

> If you're interested, you can also check how the pattern was improved through the optimization process. The “Outlier Cells” indicate the percentage of areas that did not match the original pattern after photothermal processing. In other words, if this value reaches zero, it means the simulated swell pattern perfectly matches the original image.

In [ ]:
# Plot normalized error progression over iterations
def plot_error_progression(errors, cfg):
  errors_norm = [e * 100 / cfg.resolution for e in errors]
  plt.figure(figsize=(6,4))
  plt.plot(errors_norm)
  plt.xlabel('Iteration')
  plt.ylabel('Outlier Cells [%]')
  plt.title('Optimization Error Progress')
  plt.show()

plot_error_progression(errors, cfg)

## 4. Optimize Your Pattern
**Overview:** We optimize the pattern of your choice.
0. **If you skip Sec. 3**,  Define Previous Functions
1. 📁 Upload Your Pattern
2. 🔧 Prepare for Optimization & 👀 Preview It
3. 🔁 Run Optimization
4. 👀 Save & Preview Optimized Pattern
5. 👀 Show Swell Pattern & Temperature Distribution
6. 💾 Download Optimized Pattern
7. 📉 (Optional) Show Optimization Performance

### 4-0. **If you skip Sec. 3**, Define Previous Functions  
> If you are jumping straight into this section without running the previous cells, this cell re-loads or re-defines all helper functions (like config loading, and optimization routines) so everything runs smoothly.

In [ ]:
# Locate the bundled config.json inside the installed package
def get_default_config_path():
    return pkg_resources.as_file(
        pkg_resources.files("sbl_optimizer.assets") / "config.json"
    )

# Visualize the initialized print pattern using matplotlib
def visualize_init_pattern(H_init, max_heat):
  plt.figure(figsize=(4,4))
  plt.imshow(H_init/max_heat, cmap='gray_r', vmin=0.0, vmax=1.0)
  plt.axis('off')
  plt.title("Initial print pattern")
  plt.show()

# Visualize the swell pattern and temperature distribution using matplotlib heatmap
def display_plots(plots):
  fig, (ax_swell, ax_temperature) = plt.subplots(1, 2, figsize=(4 * 2, 4))
  img_plot = Image.open(plots[0])
  ax_swell.imshow(img_plot)
  ax_swell.axis('off')
  ax_swell.set_title('Swell Pattern')
  img_plot = Image.open(plots[1])
  ax_temperature.imshow(img_plot)
  ax_temperature.axis('off')
  ax_temperature.set_title('Temperature Distribution [C°]')
  plt.tight_layout()
  plt.show()

# Ask user if they want to download the PDF locally
def download_pdf(pdf_file):
  download_choice = input('Do you want to download the optimized PDF? (y/n): ').strip().lower()
  if download_choice == 'y':
      files.download(str(pdf_file))

# Plot normalized error progression over iterations
def plot_error_progression(errors, cfg):
  errors_norm = [e * 100 / cfg.resolution for e in errors]
  plt.figure(figsize=(6,4))
  plt.plot(errors_norm)
  plt.xlabel('Iteration')
  plt.ylabel('Outlier Cells [%]')
  plt.title('Optimization Error Progress')
  plt.show()


# Load the default configuration file
with get_default_config_path() as config_path:
    with open(config_path) as f:
        cfg_data = json.load(f)
print("Default configuration:")
for k, v in cfg_data.items():
    print(f"- {k}: {v}")
# Create Config object
cfg = Config.from_file(config_path)

### 4-1. 📁 Upload Your Pattern  
> Select JPG or PNG image from your computer. Once you click Choose Files and pick your image, it will appear below.

In [ ]:
def upload_image():
  """
  Prompt the user to upload an image and save it to the working directory.
  Returns:
      Path object pointing to the uploaded image file.
  """
  uploaded = files.upload()  # Opens upload dialog
  if not uploaded:
      raise ValueError("No file uploaded.")
  # Get the first uploaded filename
  filename = next(iter(uploaded.keys()))
  print(f"Uploaded file: {filename}")
  return Path(filename)

your_img_path = upload_image()
your_img = Image.open(your_img_path)
plt.figure(figsize=(4,4))
plt.imshow(your_img)
plt.axis('off')
plt.title("Uploaded Input Image")
plt.show()

### 4-2. 🔧 Prepare for Optimization & 👀 Preview It  
> We'll first make the patterned areas of the image a uniform gray. Then, we'll adjust the image resolution to match that of the simulation. The preprocessed image is shown below. This represents the pre-optimization state.

In [ ]:
mask_yours, H_init_yours, max_heat_yours = image_to_heat_pattern(your_img_path, cfg)
visualize_init_pattern(H_init_yours, max_heat_yours)

### 4-3. 🔁 Run Optimization  
> The goal of this optimization is to create a swell pattern that closely resembles the original design, even after heat conduction occurs within the paper. If we simply make the entire pattern uniformly gray or black, the wider lines tend to overheat while the thinner ones don’t heat up enough due to the heat conduction. As a result, the temperature distribution won't match the original image, and the heat-based swells won’t either. To address this, we perform “optimization” to adjust the shading in different areas so that the resulting temperature distribution, after heat conduction, matches the original image as closely as possible.

In [ ]:
%%time
# Set the maximum height of the output
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'))

# Compute physical dimensions (for logging)
img_w_yours, img_h_yours, phys_w_yours, phys_h_yours = compute_dims(Image.open(your_img_path))

# Optimize
T_best_yours, H_best_yours, errors_yours = optimize(mask_yours, H_init_yours, max_heat_yours, img_w_yours, img_h_yours, phys_w_yours, phys_h_yours, cfg)
print(f"Optimization completed in {len(errors_yours)} iterations.")

### 4-4. 👀 Save & Preview Optimized Pattern  
> We'll now save and display the optimized pattern image. If you're satisfied with the result, you can download the image later.

In [ ]:
# Save pattern PDF
pattern_pdf_yours = save_pattern(your_img_path, H_best_yours, max_heat_yours)
print(f"Saved optimized pattern to: {pattern_pdf_yours}")

# Display the generated PDF inline in Colab
with open(pattern_pdf_yours, "rb") as pdf_file:
    encoded_pdf_yours = base64.b64encode(pdf_file.read()).decode('utf-8')
IFrame(f"data:application/pdf;base64,{encoded_pdf_yours}", width=400, height=400)

###4-5. 👀 Show Swell Pattern & Temperature Distribution

> See side-by-side visualizations of the simulated raised texture (swell pattern) and temperature across the sheet (temperature distribution).

In [ ]:
# Save and display plots with matplotlib
plots_yours = save_plots(T_best_yours, phys_w_yours, phys_h_yours, your_img_path, dpi=72)
display_plots(plots_yours)

###4-6. 💾 Download Optimized Pattern

>Finally, grab the PDF file you just created. You’re ready to print and experiment!

In [ ]:
download_pdf(pattern_pdf_yours)

###4-7. 📉 (Optional) Show Optimization Performance

> If you're interested, you can also check how the pattern was improved through the optimization process. The “Outlier Cells” indicate the percentage of areas that did not match the original pattern after photothermal processing. In other words, if this value reaches zero, it means the simulated swell pattern perfectly matches the original image.

In [ ]:
plot_error_progression(errors_yours, cfg)

##5. Change Configuration
**Overview:** We optimize the pattern of your choice with customized simulation settings (configuration).
0. **If you skip Sec. 3 & 4**,  Define Previous Functions
1. ⚙️ Customize Settings
2. 📁 Upload Your Pattern
3. 🔧 Prepare for Optimization & 👀 Preview It
4. 🔁 Run Optimization with Custom Settings
5. 👀 Save & Preview Optimized Pattern
6. 👀 Show Swell Pattern & Temperature Distribution
7. 💾 Download Optimized Pattern
8. 📉 (Optional) Show Optimization Performance

###5-0. **If you skip Sec. 3 & 4**,  Define Previous Functions
>If you are jumping straight into this section without running the Section 3 & 4, this cell re-loads or re-defines all helper functions so everything runs smoothly.

In [ ]:
# Locate the bundled config.json inside the installed package
def get_default_config_path():
    return pkg_resources.as_file(
        pkg_resources.files("sbl_optimizer.assets") / "config.json"
    )

# Visualize the initialized print pattern using matplotlib
def visualize_init_pattern(H_init, max_heat):
  plt.figure(figsize=(4,4))
  plt.imshow(H_init/max_heat, cmap='gray_r', vmin=0.0, vmax=1.0)
  plt.axis('off')
  plt.title("Initial print pattern")
  plt.show()

# Visualize the swell pattern and temperature distribution using matplotlib heatmap
def display_plots(plots):
  fig, (ax_swell, ax_temperature) = plt.subplots(1, 2, figsize=(4 * 2, 4))
  img_plot = Image.open(plots[0])
  ax_swell.imshow(img_plot)
  ax_swell.axis('off')
  ax_swell.set_title('Swell Pattern')
  img_plot = Image.open(plots[1])
  ax_temperature.imshow(img_plot)
  ax_temperature.axis('off')
  ax_temperature.set_title('Temperature Distribution [C°]')
  plt.tight_layout()
  plt.show()

# Ask user if they want to download the PDF locally
def download_pdf(pdf_file):
  download_choice = input('Do you want to download the optimized PDF? (y/n): ').strip().lower()
  if download_choice == 'y':
      files.download(str(pdf_file))

# Plot normalized error progression over iterations
def plot_error_progression(errors, cfg):
  errors_norm = [e * 100 / cfg.resolution for e in errors]
  plt.figure(figsize=(6,4))
  plt.plot(errors_norm)
  plt.xlabel('Iteration')
  plt.ylabel('Outlier Cells [%]')
  plt.title('Optimization Error Progress')
  plt.show()

# Upload image
def upload_image():
  """
  Prompt the user to upload an image and save it to the working directory.
  Returns:
      Path object pointing to the uploaded image file.
  """
  uploaded = files.upload()  # Opens upload dialog
  if not uploaded:
      raise ValueError("No file uploaded.")
  # Get the first uploaded filename
  filename = next(iter(uploaded.keys()))
  print(f"Uploaded file: {filename}")
  return Path(filename)

###5-1. ⚙️ Customize Settings

| Key | Type | Default | Description |
|-----|------|---------|-------------|
| `swell_temperature` | float | 145 [C°] | Target swelling temperature. Adjust this based on the swell temperature of the paste you have. |
| `light_power`       | float | 100.0 [W] |Light source power. Adjust this according to your light's power. |
| `light_diameter`    | float | 0.06 [m] | Diameter of the light circle on paper (m). Adjust based on the light circle diameter during heating. |
| `alpha`             | float | 5e-07 [m²/s] | Thermal diffusivity of paper. |
| `verbose`           | int   | 1 | Bool enabling logging. 0: turned off; 1: turned on. |
| `resolution`        | int   | 120000 | Number of cells paper is divided into in thermal simulations. Reduce this for faster optimization. Increase for a finer result. |

> It's best to start with the default values and see how the resulting swell pattern turns out. If it doesn’t look as expected, you can make adjustments as needed. Depending on the materials you're using, you can directly edit the numerical values in the following cell:



In [ ]:
custom_cfg_contents = {
    "swell_temperature": 140.0,
    "light_power": 120.0,
    "light_diameter": 0.08,
    "alpha": 5e-07,
    "verbose": 0,
    "resolution": 240000
}

In [ ]:
# Write to a JSON file in the notebook workspace
custom_config_path = Path("custom_config.json")
with open(custom_config_path, "w") as f:
    json.dump(custom_cfg_contents, f, indent=4)
print(f"Custom config written to: {custom_config_path}")

# Load and instantiate Config from this custom file
cfg_custom = Config.from_file(custom_config_path)
print("Loaded custom Config:")
for k, v in custom_cfg_contents.items():
    print(f"- {k}: {v}")

###5-2. 📁 Upload Your Pattern (.jpg or .png)
> Select JPG or PNG image from your computer. Once you click Choose Files and pick your image, it will appear below.

In [ ]:
your_img_path_c = upload_image()
your_img_c = Image.open(your_img_path_c)
plt.figure(figsize=(4,4))
plt.imshow(your_img_c)
plt.axis('off')
plt.title("Uploaded Input Image")
plt.show()

### 5-3. 🔧 Prepare for Optimization & 👀 Preview It  
> We'll first make the patterned areas of the image a uniform gray. Then, we'll adjust the image resolution to match that of the simulation. The preprocessed image is shown below. This represents the pre-optimization state.

In [ ]:
mask_yours_c, H_init_yours_c, max_heat_yours_c = image_to_heat_pattern(your_img_path_c, cfg_custom)
visualize_init_pattern(H_init_yours_c, max_heat_yours_c)

###5-4. 🔁 Run Optimization with Custom Settings
> The goal of this optimization is to create a swell pattern that closely resembles the original design, even after heat conduction occurs within the paper. If we simply make the entire pattern uniformly gray or black, the wider lines tend to overheat while the thinner ones don’t heat up enough due to the heat conduction. As a result, the temperature distribution won't match the original image, and the heat-based swells won’t either. To address this, we perform “optimization” to adjust the shading in different areas so that the resulting temperature distribution, after heat conduction, matches the original image as closely as possible. The simulation is based on the custom parameters you set [here](#scrollTo=jNyYGd1sY4Z_).

In [ ]:
%%time
# Set the maximum height of the output
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'))

# Compute physical dimensions (for logging)
img_w_yours_c, img_h_yours_c, phys_w_yours_c, phys_h_yours_c = compute_dims(Image.open(your_img_path_c))

# Optimize
T_best_yours_c, H_best_yours_c, errors_yours_c = optimize(mask_yours_c, H_init_yours_c, max_heat_yours_c, img_w_yours_c, img_h_yours_c, phys_w_yours_c, phys_h_yours_c, cfg_custom)
print(f"Optimization completed in {len(errors_yours_c)} iterations.")

### 5-5. 👀 Save & Preview Optimized Pattern  
> We'll now save and display the optimized pattern image. If you're satisfied with the result, you can download the image later.

In [ ]:
# Save pattern PDF
pattern_pdf_yours_c = save_pattern(your_img_path_c, H_best_yours_c, max_heat_yours_c)
print(f"Saved optimized pattern to: {pattern_pdf_yours_c}")

# Display the generated PDF inline in Colab
with open(pattern_pdf_yours_c, "rb") as pdf_file:
    encoded_pdf_yours_c = base64.b64encode(pdf_file.read()).decode('utf-8')
IFrame(f"data:application/pdf;base64,{encoded_pdf_yours_c}", width=400, height=400)

###5-6. 👀 Show Swell Pattern & Temperature Distribution

> See side-by-side visualizations of the simulated raised texture (swell pattern) and temperature across the sheet (temperature distribution).

In [ ]:
# Save and display plots with matplotlib
plots_yours_c = save_plots(T_best_yours_c, phys_w_yours_c, phys_h_yours_c, your_img_path_c, dpi=72)
display_plots(plots_yours_c)

###5-7. 💾 Download Optimized Pattern

>Finally, grab the PDF file you just created. You’re ready to print and experiment!

In [ ]:
download_pdf(pattern_pdf_yours_c)

###5-8. 📉 (Optional) Show Optimization Performance

> If you're interested, you can also check how the pattern was improved through the optimization process. The “Outlier Cells” indicate the percentage of areas that did not match the original pattern after photothermal processing. In other words, if this value reaches zero, it means the simulated swell pattern perfectly matches the original image.

In [ ]:
plot_error_progression(errors_yours_c, cfg_custom)

## 6. Troubleshooting & FAQ

**Q: What file types work?**  
A: You can upload JPG or PNG images.

**Q: My image didn’t upload.**  
A: Make sure you click the **Choose Files** button and pick a valid image. Re-run the cell if it fails.

**Q: I see an error during optimization.**  
A: Check that your file is not too large. Try a smaller image and re-run.

**Q: How long does the optimization take?**  
A: On Colab, it may take a minute or two.

**Tip:** If something goes wrong, restart the notebook (Runtime → Restart runtime) and re-run from the top.

Still having trouble? Contact

[![@RefreshSource](https://img.shields.io/badge/@RefreshSource-black?logo=x&logoColor=white)](https://x.com/refreshsource)
[![Email](https://img.shields.io/badge/sosuke.ichihashi@gmail.com-black?logo=gmail&logoColor=white)](mailto:sosuke.ichihashi@gmail.com)

## 7. Credits & License
###License
>This project is licensed under the [MIT License](https://github.com/sosucat/sbl-optimizer-colab/blob/main/LICENSE).


###Developer
>Sosuke Ichihashi

> [![Sosuke Ichihashi](https://img.shields.io/badge/Sosuke_Ichihashi-black?logo=googlescholar&logoColor=white)](https://sosuke-ichihashi.com/)
[![@sosucat](https://img.shields.io/badge/@sosucat-black?logo=github&logoColor=white)](https://github.com/sosucat)
[![@RefreshSource](https://img.shields.io/badge/@RefreshSource-black?logo=x&logoColor=white)](https://x.com/refreshsource)


###Cite this work
> If you use **sbl-optimizer** in your research or projects, please cite:
* Sosuke Ichihashi, Noura Howell, and HyunJoo Oh. 2025.\
Swell by Light: An Approachable Technique for Freeform Raised Textures. \
In Proceedings of the Nineteenth International Conference on Tangible, Embedded, and Embodied Interaction (TEI '25). Association for Computing Machinery, New York, NY, USA, Article 45, 1–16. https://doi.org/10.1145/3689050.3704420
```bibtex
@inproceedings{10.1145/3689050.3704420,
author = {Ichihashi, Sosuke and Howell, Noura and Oh, HyunJoo},
title = {Swell by Light: An Approachable Technique for Freeform Raised Textures},
year = {2025},
isbn = {9798400711978},
publisher = {Association for Computing Machinery},
address = {New York, NY, USA},
url = {https://doi.org/10.1145/3689050.3704420},
doi = {10.1145/3689050.3704420},
booktitle = {Proceedings of the Nineteenth International Conference on Tangible, Embedded, and Embodied Interaction},
articleno = {45},
numpages = {16},
keywords = {2.5D fabrication, Personal fabrication, tactile rendering},
location = {Bordeaux / Talence, France},
series = {TEI '25}
}
```

## 8. Next Steps
**Ready for more? Here’s where to explore further:**


🎨 **Fabrication Tutorial**

> See the step-by-step fabrication tutorial on our website and YouTube.

>[![Fabrication](https://img.shields.io/badge/🔗_Fabrication-black)](https://sites.gatech.edu/futurefeelings/2025/07/23/make-puffy-patterns-with-light/)
[![Watch fabrication demo on YouTube](https://img.shields.io/badge/Fabrication-750014?logo=youtube)](https://youtu.be/LomVS_jHxl0?feature=shared)


🔗 **Project Page**

> Read more about the project and example applications on our project page.

>[![Project page](https://img.shields.io/badge/🔗_Project_Page-black)](https://sites.gatech.edu/futurefeelings/2025/03/07/swell-by-light-tei-25/)


📄 **Research Paper**

> Explore various application examples of Swell by Light and learn the theory and related work behind it in our paper.

>[![Research paper](https://img.shields.io/badge/Research_Paper-black?logo=acm)](https://doi.org/10.1145/3689050.3704420)


👩‍💻 **Local Command Line Interface**

> Follow the tutorials on PyPI or GitHub to use sbl-optimizer in a simpler way on your machine.

>[![PyPI version](https://badge.fury.io/py/sbl-optimizer.svg)](https://badge.fury.io/py/sbl-optimizer) [![GitHub](https://img.shields.io/badge/sbl--optimizer-black?logo=github)](https://github.com/sosucat/sbl-optimizer)


🦙 **Simblified Colab Notebook**

> Check out this blackboxed version for future iterations.

>[![Simpler Colab](https://img.shields.io/badge/Simpler_Colab_Notebook-black?logo=googlecolab)](https://colab.research.google.com/drive/1Kpvq15wZrzsnQI28_JfkDSqCwT1ouyxj?usp=sharing)


👨‍💻 **Source Code & Issues**

> Visit our GitHub repository to report issues, contribute, or see the latest developments.

>[![GitHub sbl-optimizer](https://img.shields.io/badge/sbl--optimizer-black?logo=github)](https://github.com/sosucat/sbl-optimizer)
[![GitHub sbl-optimizer-colab](https://img.shields.io/badge/This_Notebook-black?logo=github)](https://github.com/sosucat/sbl-optimizer-colab)


🕵️‍♂️ **Author’s Website**

> Discover more work by Sosuke Ichihashi.

>[![Author](https://img.shields.io/badge/Author-black?logo=googlescholar&logoColor=white)](https://sosuke-ichihashi.com/)


*Happy fabricating!*